# 네이버 기사 분석2 - 기사 텍스트 정제
## gensim을 이용한 토픽모델링 분석
: 토픽 모델링을 적용하기 위해 텍스트를 처리해야 하는데 라이브러리인 gensim을 사용해본다. 

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [15]:
pip install gensim #gensim 설치

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install tqdm #파이썬에 있는 소스중 하나인 tqdm을 이용하여 남은 시간과 진행정도를 한눈에 확인


Note: you may need to restart the kernel to use updated packages.


In [18]:
from tqdm import tqdm_notebook #process bar
from konlpy.tag import Mecab #Mecab 등 형태소 분석기 불러오기
import string #특수문자를 클리닝에서 제거해주고 싶을 때 사용하려고 특수문자를 불러온다. 
import warnings #warning 제거

from gensim import corpora #gensim에서 사용하는 vectorizer 모듈과, LDA model을 불러온다. #corpora : data를 가져와서 gensim에서 LDA를 돌릴 수 있게 만들어주는 것
from gensim import models #models : 여기에 LDA가 있다. 

import numpy as np
import re
import pickle
import matplotlib.pyplot as  plt
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시

### 2. 텍스트 전처리 함수 만들기 
: 네이버 기사를 pickle을 사용해서 저장했으니, 불러와야하는데, 
데이터를 저번에 페이지 단위로 가져온 네이버 기사를 불러와서
1차원 리스트로 바꿔줘야 한다. .

In [23]:
def read_documents(input_file_name):
    """문서들을 주어진 이름의 파일로부터 읽어들여 돌려준다."""
    
    corpus = []
    
    with open(input_file_name, 'rb') as f: #rb : pickle파일을 읽기 위해서 read as binary
        temp_corpus = pickle.load(f) #일단 불러와야 한다. 
        
    for page in temp_corpus:
        corpus += page #page를 하나의 corpus에 다 붙여줌. 
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    cleaned_docs = []
    
    for doc in docs:
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
        cleaned_docs.append(temp_doc)

    return cleaned_docs

In [24]:
def define_stopwords(path): 
    #특수문자 추가 
    SW = set()
    # 불용어를 추가하는 방법 1.
    for i in string.punctuation: #string.puctuation : 특수 문자
        SW.add(i)

    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word)

    return SW

** 기사의 특성상 텍스트가 지저분 할 수 있다. 
그럴 때, 명사 추출방식, 즉 (키워드 추출한다는 느낌으로) 명사만 뽑아버리고 나머지를 다 버리면, 자연스럽게 특수문자가 제거된다. 
이렇듯 보통 지저분한 텍스트인 경우, 명사 추출방식을 많이 쓴다. 

In [25]:
def text_tokenizing(corpus, tokenizer):
    #명사 추출/ 형태소 분석 두 가지를 선택할 수 있게 만들어주는 햠수를 만들어보자.
    #tqdm을 이용하여 진행과정을 볼 수 있게 만들어본다. 
   
    mecab = Mecab() 
    token_corpus = []
    
     #tokenizer : 뭘 기준으로 자를 건지를 명시해 주는 변수
    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Noun Preprocessing"): #desc= : progressing bar 위에 이름테그
            token_text = mecab.nouns(corpus[n]) #corpus[n]에서 명사 추출하고, 
            token_text = [word for word in token_text if word not in SW] 
            #filtering : mecab.nouns를 통해 명사를 추출(word)했으면, 그 word가 추출한 것에 있을 때 
                
            token_corpus.append(token_text) #다 했으면, append
            
    elif tokenized == "morph": #형태소 단위
        for n in tqdm_notebook(range(len(corpus)), desc="Morph Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW]
            token_corpus.append(token_text)

    elif tokenizer == "word": #단어 단위
        for n in tqdm_notebook(range(len(corpus)), desc="Word Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW]
            token_corpus.append(token_text)
        

    return token_corpus

#함수를 불러오는 코드
input_file_name = "/Users/jeonghyeonjeong/for github/머신러닝_데이터분석A-Z_패스트캠퍼스/텍스트마이닝을 활용한 카카오톡 감성분석/naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("/Users/jeonghyeonjeong/for github/머신러닝_데이터분석A-Z_패스트캠퍼스/텍스트마이닝을 활용한 카카오톡 감성분석/stopwords-ko.txt") #불용어
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "word" or "morph" 

In [27]:
#결과 확인
print(tokenized_text)

[['본문', '내용', '플레이어', '플레이어', '오류', '우회', '함수', '추가', '발전', '자료', '과학', '기술', '정보', '통신부', '유럽', '일본', '미국', '등', '주요', '국가', '수년', '전', '인공지능', '관련', '저작', '논의', '시작', '이', '국가', '발전', '빅', '데이터', '등', '관련', '저작', '문제', '해결', '인식', '빅', '데이터', '소재', '머신', '러닝', '딥', '러닝', '방식', '학습', '특정', '결과', '구현', '기술', '이', '데이터', '수집', '저장', '처리', '과정', '데이터', '복제', '전송', '등', '과정', '필요', '빅', '데이터', '분석', '통계', '규칙', '경향', '등', '가치', '정보', '것', '텍스트', '데이터', '마이닝', '분석', '대상', '데이터', '타인', '저작물', '포함', '저작', '침해', '문제', '발생', '수', '빅', '데이터', '활용', '제한', '요소', '저작물', '활용', '저작', '동의', '빅', '데이터', '포함', '저작', '동의', '것', '사실', '가능', '시간', '비용', '소모', '때문', '영국', '프랑스', '독일', '등', '유럽', '주요', '국가', '문제', '인식', '년', '영국', '시작', '저작권법', '등', '관련', '법', '특별', '규정', '도입', '나라', '내용', '차이', '공통', '것', '상업', '연구', '목적', '경우', '저작', '동의', '저작물', '활용', '수', '점', '유럽연합', '올해', '월', '디지털', '단일', '시장', '저작', '유럽', '의회', '위원회', '지침', '을', '입법', '지침', '마련', '타인', '저작물', '활용', '은', '학문', '연구', '목적', '문화유산', '기구

### 3. 토픽 모델링에 사용할 함수들 확인하기